In [1]:
print("need a salaried job god in USA.")

need a salaried job god in USA.


In [28]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [30]:
def load_pdf_file(data):
    print(f"Loading data from: {data}")
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [31]:
extracted_data= load_pdf_file(data='/Users/veerbasantreddy/Documents/Generative-AI-Medical_bot/Data')

Loading data from: /Users/veerbasantreddy/Documents/Generative-AI-Medical_bot/Data


In [ ]:
extracted_data

In [33]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [34]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

4894


In [ ]:
pip install sentence-transformers

In [35]:
from langchain.embeddings import HuggingFaceEmbeddings

In [36]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [37]:
embeddings=download_hugging_face_embeddings()

In [38]:
query_result= embeddings.embed_query("hello")
print(len(query_result))

384


In [39]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
import os
PINECONE_API_KEY =  os.environ.get('PINECONE_API_KEY')

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as pinecone

pc = Pinecone(api_key = PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [42]:
from langchain_pinecone import PineconeVectorStore


vectorstore = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name=index_name,
    embedding=embeddings,
)